In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from bs4 import BeautifulSoup  # Import BeautifulSoup
import numpy as np  # Import numpy for NaN handling
from flask import Flask, render_template
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Load the dataset
df = pd.read_csv("Emails dataset.csv")

In [ ]:
# Remove the unnamed index column
df = df.drop(df.columns[0], axis=1)

In [ ]:
# Explore the data
print(df.head())
print(df.info())
print(df['Spam/Ham'].value_counts()) # Check the distribution of spam and ham

In [ ]:
# Convert text to lowercase
df['Message'] = df['Message'].str.lower()
df['Subject'] = df['Subject'].str.lower()

In [ ]:
def remove_html(text):
    if isinstance(text, str):
        soup = BeautifulSoup(text, 'html.parser')
        return soup.get_text()
    return text

def remove_punctuation(text):
    if isinstance(text, str):
        PUNCT_TO_REMOVE = string.punctuation
        return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    return text

def remove_stopwords(text):
    if isinstance(text, str):
        STOPWORDS = set(stopwords.words('english'))
        return " ".join([word for word in str(text).split() if word not in STOPWORDS])
    return text

df['Message'] = df['Message'].apply(remove_html)
df['Subject'] = df['Subject'].apply(remove_html)

df['Message'] = df['Message'].apply(remove_punctuation)
df['Subject'] = df['Subject'].apply(remove_punctuation)

df['Message'] = df['Message'].apply(remove_stopwords)
df['Subject'] = df['Subject'].apply(remove_stopwords)

# Handling Missing Values (NaNs) - Corrected:
df['Message'] = df['Message'].replace('', np.nan)  # Correct way to replace
df['Subject'] = df['Subject'].replace('', np.nan)  # Correct way to replace

df.dropna(subset=['Message', 'Subject'], inplace=True)  # Drop rows with NaN

In [ ]:
# Combine Subject and Message for feature extraction
df['Combined'] = df['Subject'] + ' ' + df['Message']

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['Combined'])
y = df['Spam/Ham'] # Target variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # Split data

model = MultinomialNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
# 1. Spam vs. Ham Emails (Count Plot)
plt.figure(figsize=(6, 4))
sns.countplot(x='Spam/Ham', data=df)
plt.title('Spam vs. Ham Emails')
plt.xlabel('Email Type')
plt.ylabel('Count')
plt.savefig("spam_vs_ham.png")
plt.show()

In [ ]:
# 2. Distribution of Email Lengths
df['message_length'] = df['Message'].str.len() #Use df instead of email_df
plt.figure(figsize=(8, 6))
sns.histplot(df['message_length'], kde=True) #Use df instead of email_df
plt.title('Distribution of Email Lengths')
plt.xlabel('Email Length')
plt.ylabel('Frequency')
plt.savefig("email_length_distribution.png")
plt.show()

In [ ]:
# 3. Subject Length Distribution
df['subject_length'] = df['Subject'].str.len() #Use df instead of email_df
plt.figure(figsize=(8, 6))
sns.histplot(df['subject_length'], kde=True) #Use df instead of email_df
plt.title('Distribution of Subject Lengths')
plt.xlabel('Subject Length')
plt.ylabel('Frequency')
plt.savefig("subject_length_distribution.png")
plt.show()
